In [6]:
import pandas as pd

In [9]:
path = "outputs/"
filename = "batch_profiling_results_Antonios-Laptop_cpu_20240909_121458.csv"
df = pd.read_csv(path+filename)

In [11]:
grouped_df = df.groupby(['model_alias', 'batch_size']).mean().reset_index()
grouped_df

,model_alias,batch_size,processing_time,throughput
0,distilgpt2-124m,1,0.328762,3.355
1,distilgpt2-124m,2,0.471367,4.287
2,distilgpt2-124m,4,0.574694,7.016
3,distilgpt2-124m,8,0.845346,9.567
4,distilgpt2-124m,16,1.613449,10.417
5,distilgpt2-124m,32,3.630099,9.240
6,gpt2-124m,1,0.483890,2.248
7,gpt2-124m,2,0.657598,3.069
8,gpt2-124m,4,0.974758,4.180
9,gpt2-124m,8,1.234200,6.596


In [18]:
grouped_df.loc[:,"processing_time_std"] = df.groupby(['model_alias', 'batch_size']).std().reset_index()["processing_time"]

In [19]:
grouped_df

,model_alias,batch_size,processing_time,throughput,processing_time_std
0,distilgpt2-124m,1,0.328762,3.355,0.147318
1,distilgpt2-124m,2,0.471367,4.287,0.050183
2,distilgpt2-124m,4,0.574694,7.016,0.056386
3,distilgpt2-124m,8,0.845346,9.567,0.097643
4,distilgpt2-124m,16,1.613449,10.417,0.394547
5,distilgpt2-124m,32,3.630099,9.240,0.958861
6,gpt2-124m,1,0.483890,2.248,0.199836
7,gpt2-124m,2,0.657598,3.069,0.062376
8,gpt2-124m,4,0.974758,4.180,0.150360
9,gpt2-124m,8,1.234200,6.596,0.178294


In [20]:
grouped_df.to_csv("results/batch_profiling_results_Antonios-Laptop_cpu_grouped.csv", index=False)